# Dimensionality reduction and optimization

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

np.set_printoptions(edgeitems=10, linewidth=500)

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## Load data

In [2]:
seed = 6779
rng = np.random.RandomState(seed)

In [3]:
ml_ratings_df = pd.read_csv(os.path.join("data", "movielens_small", "ratings.csv")).rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
ml_movies_df = pd.read_csv(os.path.join("data", "movielens_small", "movies.csv")).rename(columns={'movieId': 'item_id'})
ml_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')

# Filter the data to reduce the number of movies
left_ids = [1, 318, 1193, 1208, 1214, 1721, 2959, 3578, 4306, 109487]

ml_ratings_df = ml_ratings_df.loc[ml_ratings_df['item_id'].isin(left_ids)]
ml_movies_df = ml_movies_df.loc[ml_movies_df['item_id'].isin(left_ids)]
ml_df = ml_df.loc[ml_df['item_id'].isin(left_ids)]

display(ml_movies_df)

print("Number of interactions left: {}".format(len(ml_ratings_df)))

,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
896,1193,One Flew Over the Cuckoo's Nest (1975),Drama
909,1208,Apocalypse Now (1979),Action|Drama|War
915,1214,Alien (1979),Horror|Sci-Fi
1291,1721,Titanic (1997),Drama|Romance
2226,2959,Fight Club (1999),Action|Crime|Drama|Thriller
2674,3578,Gladiator (2000),Action|Adventure|Drama
3194,4306,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Ro...
8376,109487,Interstellar (2014),Sci-Fi|IMAX


Number of interactions left: 1689


## Shift item ids and user ids so that they are consecutive

In [4]:
interactions_df = ml_ratings_df.copy()

unique_item_ids = interactions_df['item_id'].unique()
item_id_mapping = dict(zip(unique_item_ids, list(range(len(unique_item_ids)))))
item_id_reverse_mapping = dict(zip(list(range(len(unique_item_ids))), unique_item_ids))
unique_user_ids = interactions_df['user_id'].unique()
user_id_mapping = dict(zip(unique_user_ids, list(range(len(unique_user_ids)))))
user_id_reverse_mapping = dict(zip(list(range(len(unique_user_ids))), unique_user_ids))

interactions_df.replace({'item_id': item_id_mapping, 'user_id': user_id_mapping}, inplace=True)

display(interactions_df.head(10))

,user_id,item_id,rating,timestamp
0,0,0,4.0,964982703
72,0,1,4.0,964983250
75,0,2,4.0,964981855
192,0,3,5.0,964983282
219,0,4,5.0,964980668
232,1,5,3.0,1445714835
235,1,4,4.0,1445714885
255,1,6,3.0,1445715145
458,2,3,2.0,945078528
516,3,0,4.0,847434962


## Get the number of items and users

In [5]:
n_items = np.max(interactions_df['item_id']) + 1
n_users = np.max(interactions_df['user_id']) + 1

print("n_items={}\nn_users={}".format(n_items, n_users))

n_items=10
n_users=521


## Get the user-item interaction matrix

In [6]:
# mapping to int is necessary because of how iterrows works
r = np.zeros(shape=(n_users, n_items))
for idx, interaction in interactions_df.iterrows():
    r[int(interaction['user_id'])][int(interaction['item_id'])] = 1
    
print(r)

[[1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 ...
 [1. 0. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]
 [1. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]]


# Dimensionality reduction

## PCA (Principal Component Analysis)

**Task 1.** Apply PCA (Principal Component Analysis) to the rows (user representations) of the interaction matrix into two dimensions. Use sklearn.decomposition.PCA. Set the transformed rows to the pca_reduced_r variable.

In [8]:
# Write your code here
import sklearn
#sklearn.decomposition.PCA
pca_reduced_r = 

### Plot the reduced data

In [ ]:
def plot_unique_datapoints_2d(data, tolerance=0.000001):
    
    max_value = np.max(np.abs(data))
    
    # Round the data so that points closer than tolerance are mapped into the same coordinates
    
    rounded_data = np.round(data, int(np.log10(1 / tolerance)))
    
    # Choose the first representative from each group mapped into the same coordinates
    
    unique_representations, indices = np.unique(rounded_data, return_index=True, axis=0)
    
    # Plot

    fig = plt.figure(tight_layout=True)
    fig.set_size_inches(16, 9)
    ax1 = fig.add_subplot(1, 1, 1)

    sns.scatterplot(x=data[indices, 0], y=data[indices, 1], ax=ax1)

    for i in indices:
        plt.text(x=data[i, 0] + max_value / 150, y=data[i, 1] + max_value / 150, s=i, 
                 fontdict=dict(color='red', size=8))

    plt.show()

In [ ]:
plot_unique_datapoints_2d(pca_reduced_r, tolerance=0.1)

### Print representations for chosen pairs of users close on the graph

Notice that their initial represenation in the interaction matrix was also similar.

In [ ]:
def print_representations(chosen_user_id_groups, representations, reduced_representations):
    for user_id_group in chosen_user_id_groups:
        for user_id in user_id_group:
            print("User {} full representation and reduced representation".format(user_id))
            print(representations[user_id])
            print(reduced_representations[user_id])
            print()
        print("==============================")

In [ ]:
chosen_user_ids = [(0, 16), (133, 52), (75, 159), (279, 346), (15, 61)]
print_representations(chosen_user_ids, r, pca_reduced_r)

### Topology of distant datapoints

**Task 2.** Write a method which returns a pair of indices of datapoints with the highest discrepancy between the distance in the reduced space and the original distance measured as:

<center>
$$
    \frac{d(x_{reduced}, y_{reduced})}{\text{max}(d(x, y), 0.001)}
$$
</center>

where $d$ is the Euclidean distance. The interface for the method:

    find_poorest_reduction(orig_data, reduced_data)

Find the pair of user ids with the highest discrepancy for the PCA reduction and set those ids to pca_user_id_1 and pca_user_id_2.

You can a loop over all pairs.

In [ ]:
# Write your code here
#tu jest błąd, mianownik i licznik powinny być odwrotnie

In [ ]:
pca_user_id_1, pca_user_id_2 = find_poorest_reduction(r, pca_reduced_r)

In [ ]:
chosen_user_ids = [(pca_user_id_1, pca_user_id_2)]
print_representations(chosen_user_ids, r, pca_reduced_r)

Interpretation: if the original vectors for the chosen pair are far away from each other, then this means that the reduction properly kept distances between points which were far away in the original space. Otherwise, it means that the reduction lost an important piece of topological information from the original space.

## tSNE

**Task 3.** Apply tSNE to the rows (user representations) of the interaction matrix into two dimensions. Use sklearn.manifold.TSNE (with init='pca'). Set the transformed rows to the tsne_reduced_r variable.

In [ ]:
# Write your code here

### Plot the reduced data

In [ ]:
plot_unique_datapoints_2d(tsne_reduced_r, tolerance=0.1)

### Print tSNE representations for chosen pairs of users close on the PCA graph

In [ ]:
chosen_user_ids = [(0, 16), (133, 52), (75, 159), (279, 346), (15, 61)]
print_representations(chosen_user_ids, r, tsne_reduced_r)

### Print tSNE representations for chosen pairs of users close on the tSNE graph

Notice that those clusters of points represent points with exactly the same interaction vector. In PCA they were mapped into the same point, but tSNE separates all points.

In [ ]:
chosen_user_ids = [(81, 39, 120, 101), (11, 49, 288, 378), (14, 329), (150, 127, 148)]
print_representations(chosen_user_ids, r, tsne_reduced_r)

### Topology of distant datapoints

**Task 4.** Find the pair of user ids with the highest discrepancy for the tSNE reduction and set those ids to tsne_user_id_1 and tsne_user_id_2.

In [ ]:
# Write your code here

In [ ]:
chosen_user_ids = [(tsne_user_id_1, tsne_user_id_2)]
print_representations(chosen_user_ids, r, tsne_reduced_r)

# Matrix factorization

## SVD (Singular Value Decomposition)

**Task 5.** Implement a method for performing SVD for an arbitrary matrix and returning the two submatrices corresponding to the $k$ biggest eigenvalues - the matrix consisting of the $k$ first columns of $P$ multiplied by the matrix consisting of the first $k$ rows and columns of $\Sigma$ (that's the first matrix in decomposition), the matrix consisting of the first $k$ columns of $Q$ (compare the lecture). For $k$ equal to None return the full decomposition. Return a tuple with both resulting matrices. 

Use numpy.linalg.svd to perform SVD. Note that this method does not return a diagonal matrix as the second matrix, but only a vector of singular values. To perform the matrix multiplication as suggested in the lecture you have to transform it into a diagonal matrix (for instance with np.diag). Note also that the $Q$ matrix returned by this method is already transposed (you have to revert the transposition). To make the calculations easier you can use this method with full_matrices=False. It will cut the matrices to the sizes $m \text{x} k$, $k \text{x} k$, $k \text{x} n$, where $m$ is the number of rows in the original matrix, $k$ is the number of non-zero eigenvalues, $n$ is the number of columns in the original matrix.

The interface of the method should have the following form:

    perform_svd(r, k=None)

To test the method decompose the matrix defined in the cell below for the following values of $k$: None, 6, 5, 4, 3, 2, 1, and then reconstruct the original matrix by multiplying the matrices from the decomposition. For every $k$ calculate and print the element-wise difference between the original matrix and the matrix retrieved from the decomposition and calculate and print the Euclidean distance between the matrices (take the the square of every element in the difference matrix, sum them up and take the square root - that's the same formula as for RMSE). You can also print the decomposition for every $k$ to see how it looks like.

Compare the decomposition given in the lecture with the one given by your method for $k = 2$ and compare the reconstructed matrices distance to the original. Which decomposition is better (i.e. gives a closer approximation)?

In [ ]:
r_aggarwal = [
    [1, 1, 1, 0, 0, 0],
    [1, 1, 1, 0, 0, 0],
    [1, 1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1, 1],
    [-1, -1, -1, 1, 1, 1],
    [-1, -1, 1, 1, 1, 1],
    [-1, -1, -1, 1, 1, 1],
]

# Write your code here

In [ ]:
# Test the method

# Write your code here

## Application of the SVD decomposition to obtain 2D representation of users for the movielens dataset subset

In [ ]:
svd_reduced_r, _ = perform_svd(r, k=2)

### Plot the reduced data

In [ ]:
plot_unique_datapoints_2d(svd_reduced_r, tolerance=0.1)

### Print representations for chosen pairs of users close on the graph

In [ ]:
chosen_user_ids = [(15, 61, 213), (65, 26, 4), (413, 304), (86, 320, 505), (43, 52, 133)]
print_representations(chosen_user_ids, r, svd_reduced_r)

### Topology of distant datapoints

In [ ]:
svd_user_id_1, svd_user_id_2 = find_poorest_reduction(r, svd_reduced_r)

In [ ]:
chosen_user_ids = [(svd_user_id_1, svd_user_id_2)]
print_representations(chosen_user_ids, r, svd_reduced_r)

# Optimization

**Task 6.** Write a method perform_sgd for performing n_steps of stochastic gradient descent (SGD) for a given alpha for a linear fit to a 2D dataset. The method should return a tuple of fitted theta_0 and theta_1. Write all the updating formulas yourself.

The interface of the method should be as follows:
    
    perform_sgd(data, theta_0_init, theta_1_init, n_steps, alpha)
    
Test several initial pairs of theta_0 and theta_1, and several values of n_steps and alpha. Does the method always converge to the same solution?

You can also compare your solution to the optimal one found by sklearn.linear_model.LinearRegression.
    
Plot the data (as scatterplot) and the fit (as lineplot) on a single seaborn chart.

In [ ]:
data = np.array([
    [0.2, 0.52],
    [0.5, 1.36],
    [0.9, 1.00],
    [1.3, 1.69],
    [1.5, 2.58],
    [2.8, 2.34]
])

# Write your code here